In [17]:
# 解決したい問題
# 「日経平均株価（NIKKEI 225）の翌日の終値が、当日の終値よりも上昇するかどうか」を予測する。

# 目的: 投資の意思決定（買いか見送りか）をサポートする。

In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [45]:
# 1. データの収集
tickers = {"N225": "^N225", "SP500": "^GSPC", "USDJPY": "JPY=X"}
raw_data = yf.download(list(tickers.values()), start="2010-01-01", end="2025-12-31")['Close']
raw_data.columns = tickers.keys()
raw_data.ffill(inplace=True)

[*********************100%***********************]  3 of 3 completed


,N225,SP500,USDJPY
Date,,,
2010-01-01,92.919998,NaN,NaN
2010-01-04,92.495003,1132.989990,10654.790039
2010-01-05,91.584999,1136.520020,10681.830078
2010-01-06,92.356003,1137.140015,10731.450195
2010-01-07,93.389999,1141.689941,10681.660156
...,...,...,...
2025-12-24,156.175995,6932.049805,50344.101562
2025-12-25,156.175995,6932.049805,50407.789062
2025-12-26,156.089996,6929.939941,50750.390625


In [46]:
# 2. 特徴量エンジニアリング
df = pd.DataFrame(index=raw_data.index)

# 日経平均の当日の騰落率（前日終値比）
df['N225_Ret'] = raw_data['N225'].pct_change()

# 【重要】前夜の米国株の騰落率（日本市場の先行指標になる）
df['SP500_Lag1'] = raw_data['SP500'].pct_change().shift(1)

# 【重要】今朝の為替の動き（先行指標）
df['USDJPY_Lag1'] = raw_data['USDJPY'].pct_change().shift(1)

# モメンタム指標：直近3日間の日経平均の累積騰落
df['N225_Mom3'] = df['N225_Ret'].rolling(window=3).sum()

# ボラティリティ（値動きの激しさ）
df['N225_Vol'] = df['N225_Ret'].rolling(window=5).std()

# ターゲット：翌日の日経平均が上昇するか(1) / 下落するか(0)
df['Target'] = (raw_data['N225'].shift(-1) > raw_data['N225']).astype(int)
df.dropna(inplace=True)

In [47]:
# 特徴量の選択（ノイズを減らすため厳選）
feature_cols = ['SP500_Lag1', 'USDJPY_Lag1', 'N225_Mom3', 'N225_Vol']
X = df[feature_cols].values
y = df['Target'].values

In [48]:
# 分割（時系列順）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [49]:
# 外れ値に強いRobustScalerを採用
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
# 3. モデルの再設計
model = Sequential([
    Dense(32, activation='leaky_relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(16, activation='leaky_relu'),
    Dense(1, activation='sigmoid')
])

c:\Users\kwata\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
# 学習率を少し高めに設定して停滞を打破
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [52]:
# 4. 学習（教師あり学習）
history = model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=32,
    validation_split=0.2,
    verbose=0 # ログを省略して結果に集中
)

In [53]:
# 5. 評価
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n==============================")
print(f"改善後のテストデータ正解率: {accuracy:.4f}")
print(f"==============================")


改善後のテストデータ正解率: 0.5228
